# Initial testing from ChatGPT

1. Install Dependencies (Colab/First Cell)

In [ ]:
#%pip install transformers datasets peft accelerate bitsandbytes
#%pip install trl xformers

2. Load Base Model + Tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)


3. Load and Format Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ybelkada/ultrachat_200k", split="train[:5000]")

def format(example):
    return {
        "text": f"<|user|> {example['prompt']} <|assistant|> {example['completion']}"
    }

dataset = dataset.map(format)

4. Set Up LoRA Adapter

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

5. Configure Trainer

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./mistral-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    learning_rate=2e-4,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

6. Train the Model

In [ ]:
trainer.train()

7. Generate with the Fine-Tuned Model

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

pipe("Tell me a joke about AI:", max_new_tokens=100)